In [ ]:
import os
import polars as pl
import plotly.express as px

In [ ]:
kaggle_run_type = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', None)
if kaggle_run_type:
    DATA_PATH = "/kaggle/input/linking-writing-processes-to-writing-quality"
else:
    DATA_PATH = "../../data"

In [ ]:
logs = pl.read_csv(f"{DATA_PATH}/train_logs.csv")
scores = pl.read_csv(f"{DATA_PATH}/train_scores.csv")

In [ ]:
logs.sample(3)

In [ ]:
scores.sample(3)

In [ ]:
essay_stats = (
    logs
    .sort("event_id")
    .group_by("id")
    .agg(
        pl.col("word_count").last(),
        (pl.col("up_time").max() - pl.col("down_time").min()).alias("write_duration"),
        pl.col("event_id").count().alias("event_count")
    )
    .with_columns(
        (pl.col("write_duration")/pl.col("event_count")).alias("frequency")
    )
    .join(scores, on="id")
)

In [ ]:
px.scatter(
    essay_stats,
    x = "write_duration",
    y = "score",
    opacity = 0.2
)

In [ ]:
px.scatter(
    essay_stats,
    x = "word_count",
    y = "score",
    opacity = 0.2
)

In [ ]:
px.scatter(
    essay_stats,
    x = "event_count",
    y = "score",
    opacity = 0.2
)

In [ ]:
px.scatter(
    essay_stats,
    x = "frequency",
    y = "score",
    opacity = 0.1
)